# CSC 480-F25 Lab 5: Games as Adversarial Search (Scrabble Agent)

### Authors:
***Arnav Bhola, Pranav Krishna***

California Polytechnic State University, San Luis Obispo;
Computer Science & Software Engineering Department

### Overview
This lab covers **Adversarial Search**, beginning with classical Minimax and Alpha-Beta pruning, then transitioning to **Monte Carlo Tree Search (MCTS)** to handle the complexity and imperfect information of Scrabble. We apply the **agentic design approach** (from Lab 2 and the heuristic integration of Lab 4) to construct a sophisticated state evaluation function that guides the MCTS playouts.

---

## Environment Setup and Imports

In [1]:
# Reusing package install from L4
%pip install "autogen-core" "autogen-agentchat" "autogen-ext[openai,azure]"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import asyncio
import time
import random
from typing import List, Tuple, Dict, Optional

# Import AutoGen classes
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

# Import core game utilities from scrabble_utils.py (must be in the same directory)
from L5_utils import ScrabbleState, AlphaBetaMinimax, MonteCarlo, Move 

### First take a look at the Scrabble Implementation

The following code randomly selects moves for both players in the Scrabble game until the game terminates.
It should give you a sense of what the API looks like.

In [8]:
# Initialize the game state
game_state = ScrabbleState.create_new_game()
print("Initial Game State:")
print(game_state)

while not game_state.is_terminal():
    current_player = game_state.current_player
    legal_moves = game_state.get_legal_moves(current_player)

    # Prefer non-pass moves if available
    non_pass_moves = [move for move in legal_moves if not move.is_pass]
    if non_pass_moves:
        chosen_move = random.choice(non_pass_moves)
    else:
        chosen_move = legal_moves[0]  # Must pass if no other
    
    game_state = game_state.apply_move(chosen_move)
    print(f"\nPlayer {current_player} plays: {chosen_move}")
    print(game_state)

Initial Game State:
SCRABBLE GAME STATE

Current Player: Player 1
Scores: Player 1: 0, Player 2: 0
Consecutive Passes: 0

Player 1 Rack: O H F S I N B (7 tiles)
Player 2 Rack: M E N A T U L (7 tiles)

Tiles Remaining in Pool: 86

Board:
Legend: · = empty, ² = 2x letter, ³ = 3x letter, * = 2x word, # = 3x word
     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
   +---------------------------------------------+
 0 | ·  ²  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ²  · |
 1 | ²  ·  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ·  ² |
 2 | ·  ·  *  ·  ·  ·  ²  ·  ²  ·  ·  ·  *  ·  · |
 3 | ·  ·  ·  #  ·  ·  ·  ²  ·  ·  ·  #  ·  ·  · |
 4 | ·  ·  ·  ·  *  ·  ·  ·  ·  ·  *  ·  ·  ·  · |
 5 | ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  · |
 6 | ·  ·  ²  ·  ·  ·  ²  ·  ²  ·  ·  ·  ²  ·  · |
 7 | ·  ·  ·  ²  ·  ·  ·  *  ·  ·  ·  ²  ·  ·  · |
 8 | ·  ·  ²  ·  ·  ·  ²  ·  ²  ·  ·  ·  ²  ·  · |
 9 | ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  · |
10 | ·  ·  ·  ·  *  ·  ·  ·  ·  ·  *  ·  ·  ·  · |
11 | ·  ·  ·  #  ·  ·  ·  ² 

### Azure OpenAI Configuration

In [9]:
from pathlib import Path
from dotenv import load_dotenv

In [10]:
cwd = Path.cwd()
env_path = cwd.parent / ".env"
loaded = load_dotenv(env_path)

In [11]:
# Reusing configuration from L4.ipynb
# Configure your Azure OpenAI client
azure_deployment = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = "2024-12-01-preview" 
azure_endpoint = os.getenv("AZURE_ENDPOINT")

# Ensure your API key is set as an environment variable
api_key = os.getenv("AZURE_SUBSCRIPTION_KEY")

client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_deployment,
    model="gpt-5-mini", # Using the default model name referenced in previous labs
    api_version=api_version,
    azure_endpoint=azure_endpoint,
    api_key=api_key,
)

---

## Part 1 & 2: Classical Search Limitations

A small note: Minimax and MCTS expect that the problem space is zero-sum. Scrabble is not inherently zero-sum with the typical method scores are calculated. However, we make it zero-sum by using the difference between the two players' scores as the metric to optimize for.

### 1. Written Analysis: Why Minimax Fails Scrabble (Part 2)

**Task:** Using the concepts of **State Space Size, Branching Factor, and Imperfect Information** (hidden opponent tiles), justify why a naive application of Minimax with Alpha-Beta pruning is impractical for Scrabble.

Scrabble presents several fundamental challenges that make classical Minimax search impractical. At any given turn, a player typically has hundreds of possible legal moves due to the 7-tile rack and 15×15 board, creating an enormous branching factor that leads to exponential node explosion even with Alpha-Beta pruning. The state space is astronomically large with 100 tiles and countless possible board configurations, far exceeding games like Chess. Most critically, Scrabble involves imperfect information since players cannot see their opponent's tiles, which violates Minimax's assumption of perfect information and forces the algorithm to either make unrealistic assumptions or search over all possible opponent tile combinations. A typical Scrabble game lasts 15 to 25 turns per player, and searching even 3 to 4 moves ahead with a branching factor of 200 would require evaluating millions of positions, making it computationally prohibitive. These factors collectively necessitate alternative approaches like Monte Carlo Tree Search that can handle large state spaces and imperfect information through sampling rather than exhaustive search.

### 2. Demonstration: Minimax Performance Limits

**Task:** Run the `AlphaBetaMinimax` on a simplified Scrabble state. Observe the exploration time and the number of nodes explored, particularly when increasing the `max_depth` to demonstrate the complexity explosion.

In [7]:
# Initialize a simple Scrabble state
initial_state = ScrabbleState.create_new_game()
print(f"Initial state created. Player 1 Rack: {initial_state.racks}")

# Experiment 1: Shallow Search (Depth 1)
print("\n--- Running Minimax (Depth 1) ---")
minimax_d1 = AlphaBetaMinimax(max_depth=1)
best_move_d1 = minimax_d1.find_best_move(initial_state)
print(f"Depth 1 suggests: {best_move_d1}")

# Experiment 2: Deeper Search (Depth 3)
print("\n--- Running Minimax (Depth 3) ---")
minimax_d3 = AlphaBetaMinimax(max_depth=3)

best_move_d3 = minimax_d3.find_best_move(initial_state)
print(f"Depth 3 suggests: {best_move_d3}")

Initial state created. Player 1 Rack: {1: ['O', 'C', 'N', 'L', 'A', 'U', 'N'], 2: ['R', 'B', 'D', 'U', 'O', 'A', 'H']}

--- Running Minimax (Depth 1) ---
Minimax search completed in: 0.0003s. Nodes explored: 77. Best move value: 20.00
Depth 1 suggests: Move(Word='CANON', Score=20, Tile Positions=[(7, 3), (7, 4), (7, 5), (7, 6), (7, 7)])

--- Running Minimax (Depth 3) ---
Minimax search completed in: 13.0200s. Nodes explored: 1845. Best move value: 14.00
Depth 3 suggests: Move(Word='CLAN', Score=12, Tile Positions=[(7, 5), (7, 6), (7, 7), (7, 8)])


---

## Part 3: Adversarial Monte Carlo Agent

Now check out MCTS with the placeholder heuristic (which currently uses the score + noise).

In [8]:
async def basic_scrabble_heuristic(state: ScrabbleState, player: int) -> float:
    # Simple heuristic: score + small random noise
    base_score = state.scores[player]
    noise = random.uniform(-0.1, 0.1)  # Small noise to break ties
    return base_score + noise


Here we're simulating a game between two players. One which uses MCTS to choose it's moves (Player 1) and the other which chooses at random (Player 2).

In [9]:
async def run_mcts_match(num_playouts: int, heuristic_fn, num_turns: int = 5):
    """Initializes and runs the Monte Carlo Agent against a simple opponent."""

    # MCTS Agent (Player 1) uses the given scrabble_heuristic
    mcts = MonteCarlo(num_playouts=num_playouts, heuristic_fn=heuristic_fn)

    current_state = ScrabbleState.create_new_game()

    print(
        f"Starting Scrabble game simulation for {num_turns} turns. MCTS Agent (P1) uses {int(num_playouts)} playouts."
    )

    for turn in range(num_turns):
        if current_state.is_terminal():
            break

        player_id = current_state.current_player

        if player_id == 1:
            # Player 1 uses MCTS
            print(f"\n--- Turn {turn+1}: Player 1 (MCTS) ---")
            move = await mcts.find_best_move(current_state)

        else:
            # Player 2 makes moves randomly (baseline)
            print(f"\n--- Turn {turn+1}: Player 2 (Random Baseline) ---")
            possible_moves = current_state.get_legal_moves(player_id)
            move = random.choice(possible_moves)

        current_state = current_state.apply_move(move)
        print(f"Player {player_id} plays: {move}")
        print(current_state)


# Task: Run a baseline MCTS match
await run_mcts_match(num_playouts=1e5, heuristic_fn=basic_scrabble_heuristic)

Starting Scrabble game simulation for 5 turns. MCTS Agent (P1) uses 100000 playouts.

--- Turn 1: Player 1 (MCTS) ---
Starting MCTS: 100000 playouts per move (for 1 possible moves)...
Player 1 plays: Pass Move
SCRABBLE GAME STATE

Current Player: Player 2
Scores: Player 1: 0, Player 2: 0
Consecutive Passes: 1

Player 1 Rack: I O E N _ O E (7 tiles)
Player 2 Rack: L K Z Y T W Q (7 tiles)

Tiles Remaining in Pool: 86

Board:
Legend: · = empty, ² = 2x letter, ³ = 3x letter, * = 2x word, # = 3x word
     0  1  2  3  4  5  6  7  8  9 10 11 12 13 14
   +---------------------------------------------+
 0 | ·  ²  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ·  ²  · |
 1 | ²  ·  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ·  ² |
 2 | ·  ·  *  ·  ·  ·  ²  ·  ²  ·  ·  ·  *  ·  · |
 3 | ·  ·  ·  #  ·  ·  ·  ²  ·  ·  ·  #  ·  ·  · |
 4 | ·  ·  ·  ·  *  ·  ·  ·  ·  ·  *  ·  ·  ·  · |
 5 | ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  ·  ·  ·  ³  · |
 6 | ·  ·  ²  ·  ·  ·  ²  ·  ²  ·  ·  ·  ²  ·  · |
 7 | ·  ·  ·  ²  ·  ·  ·  *  ·  ·  ·  ²  ·

---

## Part 4: Agentic Heuristic Design and Impact


Implement the `collaboratie_heuristic` function using AutoGen agent collaboration.

Note: Like previous labs that use an agentic heuristic function, this will take a long time to run. An agentic system for this purpose is impractical due to the overhead of making many LLM calls.

In [ ]:
async def setup_agentic_heuristic_system(client):
    """Instantiate heuristic agents for Scrabble state evaluation."""

    score_estimator = AssistantAgent(
        name="Score_Estimator",
        system_message="""You are a Scrabble scoring expert. Analyze the current game state and estimate the scoring advantage.
Consider: current score difference, board positioning, upcoming scoring opportunities.
You MUST end your response with: SCORE_EST: [number]
Example: The current position shows strong control. SCORE_EST: 35.5 """,
        model_client=client,
    )
    
    tile_quality_agent = AssistantAgent(
        name="Tile_Quality_Agent",
        system_message="""You are a Scrabble tile management expert. Analyze the tile rack quality.
Consider: vowel/consonant balance, high-value letters, potential for bingos, letter diversity.
You MUST end your response with: SCORE_TILE: [number]
Example: Good balance with common letters. SCORE_TILE: 15.0 """,
        model_client=client,
    )
    
    opponent_risk_agent = AssistantAgent(
        name="Opponent_Risk_Agent",
        system_message="""You are a Scrabble defensive strategy expert. Assess strategic risks on the board.
Consider: exposed premium squares, open lanes, opponent potential counters.
You MUST end your response with: SCORE_RISK: [number]
Example: Some exposure on triple word. SCORE_RISK: -8.0 """,
        model_client=client,
    )

    mcts_aggregator = AssistantAgent(
        name="MCTS_Aggregator",
        system_message="""You are the final evaluator. You will receive three scores:
- SCORE_EST (from Score_Estimator)
- SCORE_TILE (from Tile_Quality_Agent)  
- SCORE_RISK (from Opponent_Risk_Agent)

Calculate: Final = (SCORE_EST * 0.5) + (SCORE_TILE * 0.3) + (SCORE_RISK * 0.2)

You MUST end your response with EXACTLY: FINAL_SCORE: [number]
Example: Combining scores... FINAL_SCORE: 28.5 """,
        model_client=client,
    )

    return [score_estimator, tile_quality_agent, opponent_risk_agent], mcts_aggregator


async def collaborative_heuristic(state: ScrabbleState, player: int) -> float:
    """Use agent collaboration to evaluate board state."""
    
    from autogen_core import CancellationToken
    from autogen_agentchat.messages import TextMessage
    
    heuristic_agents, aggregator = await setup_agentic_heuristic_system(client)
    
    # Create concise state description
    opponent = 3 - player
    state_summary = f"""Player {player} Evaluation:
- Scores: P{player}={state.scores[player]} vs P{opponent}={state.scores[opponent]}
- P{player} Rack: {' '.join(state.racks[player][:7])}
- Tiles Remaining: {len(state.tile_pool)}
- Board has {sum(1 for row in state.board for cell in row if cell != '.')} tiles played"""

    try:
        # Run each agent sequentially and collect their outputs
        agent_outputs = []
        cancellation_token = CancellationToken()
        
        for agent in heuristic_agents:
            prompt = f"{state_summary}\n\nProvide your analysis as {agent.name}."
            
            # Create proper TextMessage
            message = TextMessage(content=prompt, source="user")
            
            # Run the agent with on_messages
            response = await agent.on_messages([message], cancellation_token)
            
            agent_response = response.chat_message.content
            agent_outputs.append(agent_response)
            print(f"\n[{agent.name}]: {agent_response}")
        
        # Now have aggregator combine them
        aggregator_prompt = f"""{state_summary}

Agent Analyses:
{chr(10).join(f'{i+1}. {out}' for i, out in enumerate(agent_outputs))}

Combine these scores into a final evaluation."""
        
        message = TextMessage(content=aggregator_prompt, source="user")
        response = await aggregator.on_messages([message], cancellation_token)
        
        aggregator_response = response.chat_message.content
        print(f"\n[Aggregator]: {aggregator_response}")
        
        # Parse final score
        import re
        if "FINAL_SCORE:" in aggregator_response:
            match = re.search(r'FINAL_SCORE:\s*([-+]?\d*\.?\d+)', aggregator_response)
            if match:
                score = float(match.group(1))
                print(f"Extracted score: {score}")
                return score
        
        # Try to extract from any of the responses as fallback
        all_text = aggregator_response + " ".join(agent_outputs)
        score_matches = re.findall(r'SCORE_\w+:\s*([-+]?\d*\.?\d+)', all_text)
        if score_matches:
            # Simple average as fallback
            avg_score = sum(float(s) for s in score_matches) / len(score_matches)
            print(f"Using average of agent scores: {avg_score}")
            return avg_score
            
        # Last resort
        print("Could not parse scores, using score differential")
        return float(state.scores[player] - state.scores[opponent])
        
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return float(state.scores[player] - state.scores[3-player])


await run_mcts_match(num_playouts=1, heuristic_fn=collaborative_heuristic, num_turns=1)

Starting Scrabble game simulation for 3 turns. MCTS Agent (P1) uses 1 playouts.

--- Turn 1: Player 1 (MCTS) ---
Starting MCTS: 1 playouts per move (for 17 possible moves)...

[Score_Estimator]: Score_Estimator:

Current raw difference
- Player 1 leads by 14 points (14–0).

Input consistency note
- You report “Tiles Remaining: 82” which implies only 18 tiles have been played (100‑tile set). The line “Board has 225 tiles played” is impossible in standard Scrabble; I will assume the 82 remaining figure is correct and this is an early position.

Rack quality (P1: blank, I, P, E, E, D, R)
- Strengths: two E’s and an I give good vowel coverage; the blank adds strong flexibility to form either a high‑value consonant or extend a word for hooks/bingos. P, D, R are decent scoring consonants and make many common stems (PR-, -ER, -ED).
- Weaknesses: no S or common bingogram letters (T, L, N) reduces immediate bingo chances; however the blank can substitute for one missing letter which somewhat mi

---

## Reflection & Analysis

##### What worked well in the Agentic Heuristic System?

The agentic heuristic system demonstrated several notable strengths in its implementation. The agent to agent communication was remarkably clear and structured, with each specialist agent producing detailed natural language analyses that ended with explicit numeric scores in the required format. The Score Estimator analyzed current score differences and upcoming opportunities, the Tile Quality Agent evaluated rack composition and bingo potential, and the Opponent Risk Agent assessed strategic vulnerabilities and defensive priorities. This structured output made information flow transparent and debuggable. The Aggregator agent was highly effective at synthesizing the three specialist scores using the defined weighting formula, consistently producing final scores and showing its reasoning. The modularity of the system was a major success, as each agent operated independently with its own system prompt and evaluation criteria, making the architecture easy to understand and modify. Perhaps most impressively, the quality of strategic reasoning was sophisticated. Agents provided nuanced analyses of board position, rack quality, vowel consonant balance, bingo potential, and defensive considerations that would be difficult to encode in a classical heuristic. The natural language explanations made the evaluation process interpretable and educational, revealing the thought process behind each score in ways that opaque numerical functions cannot.

##### What struggled?

Despite its strengths, the agentic system encountered significant practical challenges. The most critical issue was computational performance. Each heuristic evaluation required four sequential LLM calls, taking approximately 8 to 15 seconds per evaluation. In the MCTS context with 17 legal moves and multiple playouts, this resulted in over 15 minutes for a single game turn, making real time gameplay completely impractical. The system also occasionally struggled with parsing and score extraction. While most agent responses followed the required format, some responses buried scores within lengthy explanations or omitted them entirely, requiring fallback logic and error handling. There was some inconsistency in score magnitudes across evaluations. Different game states with similar characteristics sometimes received notably different scores, suggesting the agents lacked perfect calibration or were influenced by subtle variations in how prompts were phrased. The agents also showed a tendency toward verbose output. Each specialist produced lengthy paragraphs of analysis when only a numeric score was strictly necessary for MCTS, adding to latency without improving decision quality. Finally, there was limited coordination between agents. Each specialist worked independently without seeing other agents' analyses until the Aggregator stage, potentially missing opportunities for agents to challenge or refine each other's assessments through multi turn dialogue.

##### Agentic vs. Classical Heuristics

The agentic heuristic system demonstrates significantly greater modularity and extensibility compared to a monolithic Python function, though at a substantial computational cost. In the classical approach, all heuristic logic is embedded within a single function where scoring considerations like tile quality, board positioning, and opponent threats are manually weighted and combined by the programmer. This requires deep domain expertise and makes modifications difficult since changing one aspect may have unintended effects on others. In contrast, the agentic system distributes evaluation responsibilities across specialized agents, each focusing on a distinct strategic dimension such as score estimation, tile rack quality, or opponent risk assessment. This separation of concerns allows each agent to develop nuanced reasoning about its specific domain and communicate findings in natural language before the aggregator synthesizes them into a final score. The modularity means new strategic considerations can be added by simply introducing new agents without rewriting existing logic, and individual agents can be refined independently. However, the agentic approach is dramatically slower due to multiple LLM calls per evaluation, making it impractical for real-time game play where thousands of heuristic evaluations are needed. The classical heuristic executes in microseconds while the agentic version may take seconds per evaluation. Ultimately, the agentic system offers superior flexibility and interpretability with explicit reasoning traces, but the classical approach remains necessary for computational efficiency in actual gameplay scenarios like MCTS where speed is critical.